# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Akshara Chandrabalan

**ID**: ac2837

> **Due Date**
>
> Friday, 10/13/23, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Documents/BEE 5750 /lab03-akshara-c`


   Installed HiGHS_jll ──────────── v1.5.3+0
   Installed SnoopPrecompile ────── v1.0.3
   Installed CodecBzip2 ─────────── v0.8.0


   Installed StaticArraysCore ───── v1.4.2
   Installed BenchmarkTools ─────── v1.3.2


   Installed DiffRules ──────────── v1.15.1
   Installed Compat ─────────────── v4.10.0


   Installed CommonSubexpressions ─ v0.3.0


...

  ✓ Plots → UnitfulExt


  ✓ MathOptInterface


  ✓ HiGHS


  ✓ JuMP


  20 dependencies successfully precompiled in 328 seconds. 143 already precompiled.


In [2]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver
using Plots # plotting

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

For an example of using `JuMP.jl` to solve linear programs, see [the
relevant tutorial on the class
website](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html).

Free free to delete some of the illustrative cells and code blocks in
your notebook as you go through and solve the lab problems…this might
help reduce some potential confusion while grading about what your
answer is.

## Introduction

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
\$0.12 per bf to purchase and fir costs \$0.08 per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board. This resource
allocation problem is diagrammed in
<a href="#fig-schematic" class="quarto-xref">Figure 1</a>.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

<figure id="fig-schematic">
<img src="attachment:lab03_files/figure-ipynb/mermaid-figure-1.png" />
<figcaption>Figure 1: Flowchat of the resource allocation problem in
this lab.</figcaption>
</figure>

## Problems (10 points)

### Problem 1: Problem Formulation (5 points)

In this problem, you will go through the steps of formulating a linear
program for this problem.

#### Problem 1.1 (1 point) - done, DOUBLE CHECK

What are your decision variables? Clearly define your notation,
including what variables you are using, what they mean, and what their
units are.

ANSWER (old version): My decision variables are g1, g2 and g3, which represent the amount of lumber produced of plywood grade 1, 2 and 3, respectively, in bf/bf. g1 is as follows: $$g1 = 0.5S_1 + 1.5F_1$$ where $S_i$ is the amount of spruce input in bf for grade i = 1, 2, 3 and $F_i$ is the amount of fir input in bf for grade i = 1, 2, 3. Accordingly, $$g2 = 1S_2 + 2F_2$$ and $$g3 = 1.5S_3 + 2F_3$$ 

ANSWER (what i'm currently going with): My decision variables are $S_i$ and $F_i$. $S_i$ is the amount of spruce input in bf for plywood grade i = 1, 2, 3 and $F_i$ is the amount of fir input in bf for plywood grade i = 1, 2, 3. Based on $S_i$ and $F_i$, the amount of lumber produced of plywood grade 1, 2 and 3 (i.e. g1, g2, and g3), respectively, is: $$g1 = 0.5S_1 + 1.5F_1$$ $$g2 = 1S_2 + 2F_2$$ $$g3 = 1.5S_3 + 2F_3$$     

#### Problem 1.2 (1 point) - done, DOUBLE CHECK

Derive your objective function. Support your function with
justifications and/or equations as necessary. You will not receive
credit just for the function alone.

ANSWER: The goal is to maximize profit $P$ ($/bf), so profit is the sum of the revenue from each grade of plywood minus the cost associated with producing each grade of plywood. As such, the objective function is, $$P = [(0.5S_1 + 1.5F_1)(400/1000) - (0.12S_1 + 0.08F_1)] + [(1S_2 + 2F_2)(520/1000) - (0.12S_2 + 0.08F_2)] + [(1.5S_3 + 2F_3)(700/1000) - (0.12S_3 + 0.08F_3)]$$ After expanding and simplifying, $$P = 0.13S_1 + 0.4S_2 + 0.93S_3 + 0.52F_1 + 0.96F_2 + 1.32F_3$$ I divided the revenue by 1000 to get the dollar amount per bf.

#### Problem 1.3 (2 point)

Derive any needed constraints. Support your function with justifications
and/or equations as necessary. You will not receive credit just for the
final constraints alone.

ANSWER: The maximum amount of spruce available from the forest is 320,000 bf and for fir, 720,000 bf. So, the total amount of spruce and fir used cannot exceed those limits. As such, the material constraints for spruce and fir are: $$S_1 + S_2 + S_3 \leq 320000$$ $$F_1 + F_2 + F_3 \leq 720000$$ Non-negativity constraint because the amount of spruce and fir used cannot be less than 0 (negative amount of wood doesn't make sense): $$S_i, F_i \geq 0$$ 

#### Problem 1.4 (1 point) - done, DOUBLE CHECK

Put this optimization problem in mathematical programming form. For an
example of the syntax for this, see lines 82–91
[here](https://github.com/vsrikrish/environmental-systems-analysis/blob/Fall23/tutorials/julia-jump.qmd).

 $$ \begin{aligned}
    \max_{S_i, F_i}\quad & 0.13S_1 + 0.4S_2 + 0.93S_3 + 0.52F_1 + 0.96F_2 + 1.32F_3\\
    \text{Subject to} \quad & S_1 + S_2 + S_3 \leq 320000\\
     & F_1 + F_2 + F_3 \leq 720000\\
     & S_1 \geq 0\\
     & S_2 \geq 0\\
     & S_3 \geq 0\\
     & F_1 \geq 0\\
     & F_2 \geq 0\\
     & F_3 \geq 0\\
    \end{aligned} $$

### Problem 2: Find the Solution (5 points)

#### Problem 2.1 (2 points)

Code your linear program using `JuMP`. Feel free to consult [the
website’s `JuMP`
tutorial](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html)
for syntax help. The keys:

In [16]:
lumber_model = Model(HiGHS.Optimizer)

I = 1:3
@variable(lumber_model, S[i in I] >= 0)
@variable(lumber_model, F[i in I] >= 0)

@constraint(lumber_model, material_spruce, sum(S) <= 320000)
@constraint(lumber_model, material_fir, sum(F) <= 720000)

@objective(lumber_model, Max, 0.13S[1] + 0.4S[2] + 0.93S[3] + 0.52F[1] + 0.96F[2] + 1.32F[3])

@expression(lumber_model, g1, 0.5S[1] + 1.5F[1])
@expression(lumber_model, g2, 1S[2] + 2F[2])
@expression(lumber_model, g3, 1.5S[3] + 2F[3])

print(lumber_model)


Max 0.13 S[1] + 0.4 S[2] + 0.93 S[3] + 0.52 F[1] + 0.96 F[2] + 1.32 F[3]
Subject to
 material_spruce : S[1] + S[2] + S[3] ≤ 320000
 material_fir : F[1] + F[2] + F[3] ≤ 720000
 S[1] ≥ 0
 S[2] ≥ 0
 S[3] ≥ 0
 F[1] ≥ 0
 F[2] ≥ 0
 F[3] ≥ 0


1.  Initialize your model with a solver; in this case, we’ll use the
    `HiGHS` solver, but there are other solvers listed here for
    different types of problems, some of which are open and some of
    which require a commercial license:
    <https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers>:

    ``` julia
    example_model = Model(HiGHS.Optimizer)
    ```

2.  Define variables with syntax like

    ``` julia
    @variable(example_model, 1 >= example_x >= 0)
    ```

    This will create a variable `example_x` which is constrained to be
    between 0 and 1; you can leave off any of the bounds if a variable
    is unbounded in a particular direction. You can also add a vector of
    variables:

    ``` julia
    T = 1:3 # define set to index variables
    @variable(example_model, 1 >= example_z[t in T] >= 0)
    ```

    which will create a vector of 3 variables `example_z[1]`, …,
    `example_z[3]`, all of which are bounded between 0 and 1.

3.  Add an objective with

    ``` julia
    @objective(example_model, Max, example_x + sum(example_z))
    ```

    which will add an objective to maximize (replace with `Min` to
    minimize).

4.  Add constraints:

    ``` julia
    @constraint(example_model, constraint1, 2example_x + 3*sum(example_z) <= 10)
    @constraint(example_model, constraint2, 5example_x - example_z[1] <= 2)
    ```

    which will name the constraints `constraint1` and `constraint2` (you
    should make yours more descriptive about what the constraint
    actually is). The value of adding a name is to facilitate later
    querying of shadow prices, which we will discuss later. You can also
    add a vector of constraints which have similar structure or rely on
    different elements of a data vector:

    ``` julia
    A = [2; 4]
    b = [8; 12]
    I = 1:2 # set indices for constraint
    @constraint(example_model, vector_constraint[i in I], A[i] * sum(example_z) .<= b[i])
    ```

    You can also define matrices of constraints which depend on two
    index sets by generalizing this syntax, e.g.

    ``` julia
    @constraint(example_model, matrix_constraint[i in I, j in J, ...])
    ```

    > **Tip**
    >
    > Specifying higher-dimensional vectors and matrices of variables
    > and constraints will be important when we start looking at more
    > complex applications, so don’t skip over this! You don’t want to
    > manually enter thousands of constraints to ensure hourly
    > electricity demand is met…

    Finally, you can (and probably should) `print` your model to make
    sure that you get something that looks like the equations that you
    wrote down (in a notebook, this will be nicely rendered):

    ``` julia
    print(example_model)
    ```

    $$ \begin{aligned}
    \max\quad & example\_x + example\_z_{1} + example\_z_{2} + example\_z_{3}\\
    \text{Subject to} \quad & 2 example\_x + 3 example\_z_{1} + 3 example\_z_{2} + 3 example\_z_{3} \leq 10\\
     & 5 example\_x - example\_z_{1} \leq 2\\
     & 2 example\_z_{1} + 2 example\_z_{2} + 2 example\_z_{3} \leq 8\\
     & 4 example\_z_{1} + 4 example\_z_{2} + 4 example\_z_{3} \leq 12\\
     & example\_x \geq 0\\
     & example\_z_{1} \geq 0\\
     & example\_z_{2} \geq 0\\
     & example\_z_{3} \geq 0\\
     & example\_x \leq 1\\
     & example\_z_{1} \leq 1\\
     & example\_z_{2} \leq 1\\
     & example\_z_{3} \leq 1\\
    \end{aligned} $$

    > **Define your entire model in one cell**
    >
    > `JuMP` has great and intuitive syntax, but it doesn’t like
    > re-defining variables or constraints once they’ve been set. I
    > recommend putting all of your model-definition code (starting from
    > `model = Model(...)`) for a particular optimization problem in a
    > single notebook cell, so you can re-set up the entire problem with
    > a single click when you want to make a change.

#### Problem 2.2 (1 points) - done, DOUBLE CHECK

Find the solution to your program and find the optimal values of the
decision variables. Once you’ve defined your model, you can find the
solution with \`optimize!():

In [18]:
optimize!(lumber_model)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 6 cols, 6 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-6); elements 0(-6) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.2480000000e+06
HiGHS run time      :          0.00


> **What if I Get An Error?**
>
> If `optimize!()` throws an error, that’s usually a sign that something
> is wrong with the formulation (for example, a variable might not be
> bounded or a constraint might not be specified correctly) or a typo in
> the model definition. Linear programs should be well behaved!

To find the values of variables after optimizing, use `value.()` (the
broadcasting ensures this will work for vector-valued variables as
well):

In [19]:
value.(S)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
      0.0
      0.0
 320000.0

In [20]:
value.(F)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
      0.0
      0.0
 720000.0

In [21]:
value.(g1)

0.0

In [22]:
value.(g2)

0.0

In [23]:
value.(g3)

1.92e6

ANSWER: The optimal solution is: $S_1$ = 0 bf, $S_2$ = 0 bf, $S_3$ = 320,000 bf, $F_1$ = 0 bf, $F_2$ = 0 bf, and $F_3$ = 720,000 bf. Therefore, to maximize revenue, 1,920,000 bf/bf of grade 3 plywood should be produced. The maximum profit is $1,248,000. 

#### Problem 2.3 (1 point) - done, DOUBLE CHECK

How would your profit change if you could buy 1,000 additional bf of
spruce? You can answer this by getting the shadow price of a particular
variable with:

In [32]:
profit_change_spruce = shadow_price(material_spruce) * 1000 # this is why we named the constraints when we defined them


930.0

ANSWER: The profit would increase by $930.

#### Problem 2.4 (1 point) - done, DOUBLE CHECK

Would you prefer to have 2,000 additional bf of spruce or 1,000
additional bf of fir?

In [33]:
profit_change_fir = shadow_price(material_fir) * 1000


1320.0

In [53]:
profit_spruce_2000 = profit_change_spruce * 2
profit_fir_1000 = profit_change_fir * 1

@show profit_spruce_2000
@show profit_fir_1000;


profit_spruce_2000 = 1860.0
profit_fir_1000 = 1320.0


ANSWER: I would prefer to have 2000 additional bf of spruce over 1000 additional bf of fir because the fromer results in profit increase of $1,860, while the later results in a lower profit increase of $1,320.

## References - COMPLETE

Put any consulted sources here, including classmates you worked with/who
helped you.

LEC 16

https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html

https://viveks.me/environmental-systems-analysis/tutorials/latex-notebook.html
